## Connecting to postgreSQL

In [12]:
import os
tmp = os.getcwd()
os.chdir(tmp.split("Coding")[0] + "Coding")

In [13]:
from dotenv import load_dotenv

In [14]:
load_dotenv(".env")

True

In [15]:
import psycopg2
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import sqlalchemy

In [16]:
tmp = os.getcwd()
os.chdir(tmp.split("Coding")[0] + "Coding/jmspack")
from jmspack.internal_utils import (postgresql_data_extraction,
                                    postgresql_table_names_list)

In [17]:
# def postgresql_data_extraction(sql_query: str="SELECT * from suggested_energy_intake"):
#     df = pd.DataFrame()
#     try:
#         conn = psycopg2.connect(host=os.getenv("postgresql_host"),
#                                 database="tracker",
#                                 user="tracker",
#                                 password=os.getenv("postgresql_password"))
#         df = pd.read_sql_query(sql_query, conn)
#         _ = conn.close()
#     except:
#         print("I am unable to connect to the database")
    
#     return df

In [37]:
postgresql_table_names_list()

[('django_session',),
 ('suggested_energy_intake',),
 ('django_migrations',),
 ('django_content_type',),
 ('auth_permission',),
 ('auth_group',),
 ('auth_group_permissions',),
 ('auth_user_groups',),
 ('auth_user_user_permissions',),
 ('auth_user',),
 ('django_admin_log',)]

In [38]:
df = postgresql_data_extraction(table_name = 'iris_test')

I am unable to connect to the database


In [39]:
# def aws_data_extraction(filepath: str, bucket="neurocast-vm-shared", **kwargs):
#     s3 = boto3.client(
#             "s3",
#             aws_access_key_id=os.getenv("AWS_KEY"),
#             aws_secret_access_key=os.getenv("AWS_SECRET_KEY"),
#         )
#     obj = s3.get_object(Bucket=bucket, Key=filepath)

#     return pd.read_csv(obj["Body"], **kwargs)

In [40]:
df

""


In [ ]:
stop!

### Making the initial connection to the database

In [21]:
try:
    conn = psycopg2.connect(host="188.166.98.139",
                            database="tracker",
                            user="tracker",
                            password="HAA5oJGgylRToGIrpEKc")
except:
    print("I am unable to connect to the database") 

In [24]:
def delete_postgresql_table(database_name: str,
                             user: str,
                            table_name: str,
                            ):
    try:
        conn = psycopg2.connect(
            host=os.getenv("postgresql_host"),
            database=database_name,
            user=user,
            password=os.getenv("postgresql_password"),
        )
        cursor = conn.cursor()
        _ = cursor.execute("DROP TABLE iris_test;")
        _ = conn.commit()
        _ = conn.close()
    except psycopg2.Error as e:
        print(e)
    
    return f"{table_name} has been deleted from {database_name}"

In [36]:
delete_postgresql_table(database_name="tracker",
                             user="tracker",
                            table_name="iris_test",)

'iris_test has been deleted from tracker'

In [ ]:
cursor = conn.cursor()
cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
print(cursor.fetchall())
table_list = cursor.fetchall()
list(cursor.fetchall())

In [ ]:
def postgresql_table_names_list(postgresql_conn: psycopg2.connect):
    table_list = False
    try:
        cursor = postgresql_conn.cursor()
        cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
        table_list = cursor.fetchall()
#         print(exists)
        cursor.close()
    except psycopg2.Error as e:
        print(e)
    return table_list

In [ ]:
temp = postgresql_table_names_list(postgresql_conn=conn)

In [ ]:
temp

### Read in data from suggested_energy_intake table from postgreSQL database

In [ ]:
df = pd.read_sql_query("SELECT * from suggested_energy_intake", conn)

# Verify that result of SQL query is stored in the dataframe
display(df.head(1))

conn.close()

In [ ]:
df.head()

In [ ]:
conn.close()

In [ ]:
conn

In [ ]:
df_dtypes_dict = df.dtypes.to_dict()

In [ ]:
python_to_sql_dtypes_dict = {"object":"text",
"float64":"float",
"float32": "float",
"int8": "int",
"int32": "int", 
 "int64": "int",
"datetime64[ns]": "timestamp"}

In [ ]:
columns_dtypes_list = [f"{column} {python_to_sql_dtypes_dict[str(df_dtypes_dict[column])]}" for column in df.columns.tolist()]

In [ ]:
table_name = "suggested_energy_intake"
columns_dtypes_str = ',\n'.join(columns_dtypes_list)
print(f"""CREATE TABLE {table_name} (
            {columns_dtypes_str}
)""")

In [25]:
def create_table_based_on_df(df: pd.DataFrame,
                             database_name: str,
                             user: str,
                            table_name: str,
                            ):
    python_to_sql_dtypes_dict = {"object":"text",
                                "float64":"float",
                                "float32": "float",
                                "int8": "int",
                                "int32": "int", 
                                 "int64": "int",
                                "datetime64[ns]": "timestamp"}
    df_dtypes_dict = df.dtypes.to_dict()
    columns_dtypes_list = [f"{column} {python_to_sql_dtypes_dict[str(df_dtypes_dict[column])]}" for column in df.columns.tolist()]
    columns_dtypes_str = ',\n'.join(columns_dtypes_list)
    table_string = f"""CREATE TABLE {table_name} (
                {columns_dtypes_str}
    )"""
    
    try:
        conn = psycopg2.connect(
            host=os.getenv("postgresql_host"),
            database=database_name,
            user=user,
            password=os.getenv("postgresql_password"),
        )
        cursor = conn.cursor()
        _ = cursor.execute(table_string)
        _ = conn.commit()
        _ = conn.close()
    except psycopg2.Error as e:
        print(e)
    
    return table_string

In [27]:
import seaborn as sns

df = sns.load_dataset("iris")

In [29]:
create_table_based_on_df(df=df,
                         database_name="tracker",
                             user="tracker",
                            table_name="iris_test",
                            )

'CREATE TABLE iris_test (\n                sepal_length float,\nsepal_width float,\npetal_length float,\npetal_width float,\nspecies text\n    )'

In [32]:
add_data_to_sql_table(df=df,
                         database_name="tracker",
                             user="tracker",
                            table_name="iris_test",
                            )

'INSERT INTO iris_test (sepal_length, sepal_width, petal_length, petal_width, species) VALUES (%s, %s, %s, %s, %s)'

In [ ]:
df.head(1)

In [ ]:
# for oid in range(0, df.shape[0]):
#     for col_number in range(0, df.shape[1]):
#         print(df.loc[oid, df.columns[col_number]])

In [ ]:
for oid in range(0, df.shape[0]):
    value_tuple = tuple(df.loc[oid, df.columns[col_number]] for col_number in range(0, df.shape[1]))
#     print(value_tuple)

In [31]:
def add_data_to_sql_table(df: pd.DataFrame,
                             database_name: str,
                             user: str,
                            table_name: str,
                            ):
    
    columns_string = ', '.join(df.columns.tolist())
    value_placeholder_string = ", ".join(["%s" for x in range(0, df.shape[1])])
    insert_string = f"""INSERT INTO {table_name} ({columns_string}) VALUES ({value_placeholder_string})"""
    try:
        conn = psycopg2.connect(
            host=os.getenv("postgresql_host"),
            database=database_name,
            user=user,
            password=os.getenv("postgresql_password"),
        )
        cursor = conn.cursor()
        for oid in range(0, df.shape[0]):
            value_tuple = tuple(df.loc[oid, df.columns[col_number]] for col_number in range(0, df.shape[1]))
            _ = cursor.execute(insert_string, value_tuple)
        
        _ = conn.commit()
        _ = conn.close()
        
    except psycopg2.Error as e:
        print(e)
    
    return insert_string

In [ ]:
columns_string = ', '.join(df.columns.tolist())
value_placeholder_string = ", ".join(["%s" for x in range(0, df.shape[1])])

In [ ]:
insert_string = f"""INSERT INTO {table_name} ({columns_string}) VALUES ({value_placeholder_string})""", 

In [ ]:
c = conn.cursor()
try: 
    for oid in range(0, df.shape[0]):
#         print(oid)
        c.execute("""INSERT INTO suggested_energy_intake (nutrient, daily_requirements, units) VALUES (%s, %s, %s)""", 
        (df.loc[oid, "Nutrient"], df.loc[oid, "Daily requirements"], df.loc[oid, "Units"]))
except:
    print("Unable to upload data to table")
conn.commit() # <--- makes sure the change is shown in the database
conn.close()
c.close()

### Creating a new table in the database

In [ ]:
# c = conn.cursor()
# try:
#     # Create table
#     c.execute("""CREATE TABLE suggested_energy_intake (
#             nutrient text,
#             daily_requirements decimal,
#             units text
#             )""")
# #     cur.execute("CREATE TABLE test (id serial PRIMARY KEY, num integer, data varchar);")
# except:
#     print("I can't drop our test database!")

# conn.commit() # <--- makes sure the change is shown in the database
# conn.close()
# c.close()

### Uploading data to an already created table

In [ ]:
# c = conn.cursor()
# try: 
#     for oid in range(0, df.shape[0]):
# #         print(oid)
#         c.execute("""INSERT INTO suggested_energy_intake (nutrient, daily_requirements, units) VALUES (%s, %s, %s)""", 
#         (df.loc[oid, "Nutrient"], df.loc[oid, "Daily requirements"], df.loc[oid, "Units"]))
# except:
#     print("Unable to upload data to table")
# conn.commit() # <--- makes sure the change is shown in the database
# conn.close()
# c.close()

In [ ]:
stop!

### Local SQLite database connection - read in data

In [ ]:
con = sqlite3.connect("energy_tracker.db")
df = pd.read_sql_query("SELECT * from suggested_energy_intake", con)

# Verify that result of SQL query is stored in the dataframe
display(df.head(1))

con.close()

In [ ]:
df.columns.tolist()